# Author: Faolán Hamilton
## Project Description: Analyse the planning applications submitted to Clare County Council over the past year 
### Source - https://planning.geohive.ie/datasets/housinggovie::irishplanningapplications/explore?layer=0&showTable=true

# Strategy
- Filter the dataset to only Clare County Council (>500,000 records down to 11,000)
- Filter the dataset to only the past year (2025 inclusive ~ 1,000)
- Filter out unneccessary headings 

## Clean-up
- Eircode out of address using regex
- Can I use commas  to determine a townland for address and group by settlement?

# Potential Questions:
- How many of the planning applications are granted?
- Determine a type for each (Residential, commercial etc.)
- Number of residential units 

In [1]:
#Modules to import 
import pandas as pd

In [2]:
df = pd.read_csv("IrishPlanningApplications_-505265856342584481.csv")

In [3]:
headers = df.columns.tolist()
headers

['OBJECTID',
 'Planning Authority',
 'Application Number',
 'Development Description',
 'Development Address',
 'Development Postcode',
 'ITMEasting',
 'ITMNorthing',
 'Application Status',
 'Application Type',
 'Applicant Forename',
 'Applicant Surname',
 'Applicant Address',
 'Decision',
 'Land Use Code',
 'AreaofSite',
 'NumResidentialUnits',
 'One-Off House',
 'FloorArea',
 'ReceivedDate',
 'WithdrawnDate',
 'DecisionDate',
 'DecisionDueDate',
 'GrantDate',
 'ExpiryDate',
 'Appeal Reference Number',
 'Appeal Status',
 'Appeal Decision',
 'AppealDecisionDate',
 'AppealSubmittedDate',
 'FIRequestDate',
 'FIRecDate',
 'Link Application Details',
 'One-Off KPI',
 'ETL_DATE',
 'Site Id',
 'ORIG_FID',
 'x',
 'y']

In [4]:
#remove the headers that are not relevant
drop_col_list = ['OBJECTID','Planning Authority','ITMEasting','ITMNorthing','Applicant Forename','Applicant Surname','Applicant Address','WithdrawnDate','GrantDate','ExpiryDate','Appeal Reference Number','Link Application Details','AppealSubmittedDate','FIRequestDate','FIRecDate', 'Appeal Status', 'Site Id', 'ETL_DATE', 'ORIG_FID']
df.drop(columns=drop_col_list, inplace=True)
df.head(1)

,Application Number,Development Description,Development Address,Development Postcode,Application Status,Application Type,Decision,Land Use Code,AreaofSite,NumResidentialUnits,One-Off House,FloorArea,ReceivedDate,DecisionDate,DecisionDueDate,Appeal Decision,AppealDecisionDate,One-Off KPI,x,y
0,2560001,of the development which consists of: a) Alt...,"Ardcloony , Killaloe , Co. Clare",NaN,APPLICATION FINALISED,RETENTION,CONDITIONAL,NaN,0.932,0,NaN,0.0,1/2/2025 12:00:00 AM,4/9/2025 12:00:00 AM,4/9/2025 12:00:00 AM,NaN,NaN,No,-943599.469305,6.942243e+06
